In [ ]:
# default_exp core

# covid

> Utility methods to process the John Hopkins University Covid-19 dataset using [nbdev](http://nbdev.fast.ai/).

In [ ]:
#hide
from nbdev.showdoc import *

This module has a dependency on the following Python libraries which have been added to the `requirements` field in `settings.ini`:
* `requests`
* `pandas`
* `matplotlib`
* `seaborn`

In [ ]:
#export
import typing
from typing import List, Callable
import requests
import datetime
from datetime import date
import io
import os
from io import StringIO
import urllib.request
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import altair as alt
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) # for handling log(0)

ALTAIR_W = 750
ALTAIR_H = 375

alt.renderers.enable('svg')

RendererRegistry.enable('svg')

The following cell contains some defaults for `seaborn`:

In [ ]:
#export
def setDefaults(figsize=(18,9)):
    sns.set_style("dark")
    sns.set(rc={'legend.fontsize':14,
                'xtick.labelsize':14,
                'ytick.labelsize':14,
                'axes.labelsize':16,
                'axes.titlesize':18,
                'figure.figsize':figsize,
               })

The following cell contains the names of the time series files published by John Hopkins University (JHU).  Note that the format of these names changed overnight on 24.03.20 without prior warning.

In [ ]:
#export
ROOT      = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data'
CONFIRMED = 'time_series_covid19_confirmed_global.csv'
DEATHS    = 'time_series_covid19_deaths_global.csv'
RECOVERED = 'time_series_19-covid-Recovered.csv'

## 1. Graphing current counts

The following cell contains a couple of utility functions for returning yesterday and today as strings:

In [ ]:
#export
getToday: Callable[[None], str] = lambda: date.today().strftime('%m-%d-%Y')
getYesterday: Callable[[None], str] = lambda: (date.today() - datetime.timedelta(days = 1)).strftime('%m-%d-%Y')
getDayBeforeYesterday: Callable[[None], str] = lambda: (date.today() - datetime.timedelta(days = 2)).strftime('%m-%d-%Y')

The following two utility functions are used to process a JHU csv file and turn it into a `pandas` dataframe.  Note that the dataset changed a couple of column names on 24.03.20.

In [ ]:
#export
def procDataframe(csv: str) -> pd.DataFrame:
    ''' Convert input csv data or file to a pandas dataframe. '''
    assert(csv)
    df = pd.read_csv(csv)
    if df.empty == True:
        print(f'Empty data frame - are you sure the file/url exists?')
        return df
    try:
        df['Province/State'].fillna('',inplace=True)
    except:
        df['Province_State'].fillna('',inplace=True)
    df.fillna(0, inplace=True)
    cols = df.columns.to_list()
    if 'Last Update' in cols:
        df['Last Update'] = df['Last Update'].apply(pd.to_datetime)
    if 'Last_Update' in cols:
        df['Last_Update'] = df['Last_Update'].apply(pd.to_datetime)
    return df

def procUrl(url: str, download: bool, localfile: str=None, force: bool=False, verbose: bool=False) -> pd.DataFrame:
    ''' Optionally download then process csv data or file at url converting it to a pandas dataframe. '''
    assert(url)
    if download:
        if os.path.exists(localfile) and not force:
            verbose and print(f'"{localfile}" already exists so will not overwrite')
        else:
            verbose and print(f'Downloading "{localfile}" from "{url}"...')
            urllib.request.urlretrieve(url, localfile)
        return procDataframe(localfile)
    else:
        s = requests.get(url).content
        return procDataframe(io.StringIO(s.decode('utf-8')))

The following cell contains a utility method to return a `pandas` dataframe with a given day's daily report and then ploty it by `kind` which can be one of `["Confirmed","Deaths","Recovered"]`:

In [ ]:
#export
def getCountriesDailyReport(day: str, download: bool=False, force: bool=False) -> pd.DataFrame:
    assert(len(day))
    url = f'{ROOT}/csse_covid_19_daily_reports/{day}.csv'
    localfile = f'{day}.csv'
    return procUrl(url, download, localfile, force)

def removeGrid(chart: alt.Chart) -> alt.Chart:
    return chart.\
    configure_axis(
        grid=False
    ).configure_view(
        strokeWidth=0
    )

def plotCountriesDailyReport(df: pd.DataFrame, which: str, topN: int=10, color: str='y',
                             kind: str='Confirmed', grid: bool=True, visualisation: str='matplotlib') -> None:
    if df.empty == True:
        print(f'Empty data frame - are you sure the file/url exists?')
        return df
    if visualisation == 'altair':
        sdf = df.groupby('Country_Region')[kind].sum().sort_values(ascending=False)[:topN]
        keys = sdf.keys().to_list()
        vals = sdf.to_list()
        data = []
        for k,v in zip(keys,vals):
            data.append({'Country':k, kind: v})
        sdf = pd.DataFrame(data)
        bar_chart = alt.Chart(sdf).mark_bar(size=30).encode(
            x=alt.X('Country',sort='-y'),
            y=kind,
            color=alt.value(color),
            opacity=alt.value(0.9),
        ).properties(
            width=ALTAIR_W,
            height=ALTAIR_H,
            title=f'Total {kind} by top {topN} countries as of {which}'
        )
        if not grid:
            bar_chart = removeGrid(bar_chart)
        return bar_chart
    else:
        fig, ax = plt.subplots()
        ax = df.groupby('Country_Region')[kind].sum().sort_values(ascending=False)[:topN].\
          plot(ax=ax, kind='bar', color=color, stacked=False, figsize=(18,9))
        ax.set_ylabel(kind, size=14)
        ax.set_xlabel('Country', size=14)
        ax.set_title(f'Total {kind} by top {topN} countries as of {which}', size=18)
        plt.show()

def plotCountryDailyReport(df: pd.DataFrame, country: str, which: str, topN: int=10, color: str='orange',
                           kind: str='Confirmed', grid: bool=True, visualisation: str='matplotlib') -> None:
    if df.empty == True:
        print(f'Empty data frame - are you sure the file/url exists?')
        return df
    if visualisation == 'altair':
        sdf = df[df['Country_Region'] == country].groupby('Province_State')[kind].sum().sort_values(ascending=False)[:topN]
        keys = sdf.keys().to_list()
        vals = sdf.to_list()
        data = []
        for k,v in zip(keys,vals):
            data.append({'Province_State':k, kind: v})
        sdf = pd.DataFrame(data)
        bar_chart = alt.Chart(sdf).mark_bar(size=30).encode(
            x=alt.X('Province_State',sort='-y'),
            y=kind,
            color=alt.value(color),
            opacity=alt.value(0.9),
        ).properties(
            width=ALTAIR_W,
            height=ALTAIR_H,
            title=f'Total {kind} for top {topN} regions of {country} as of {which}'
        )
        if not grid:
            bar_chart = removeGrid(bar_chart)
        return bar_chart
    else:
        ax = df[df['Country_Region'] == country].groupby('Province_State')[kind].sum().sort_values(ascending=False)[:topN].\
          plot(kind='bar', color=color, stacked=False, figsize=(18,9))
        ax.set_ylabel(kind, size=14)
        ax.set_xlabel('Country', size=14)
        ax.set_title(f'Total {kind} for top {topN} regions of {country} as of {which}', size=18)

## 2. Graphing time series counts

The following cell contains three utility methods to return time series data for each of `["Confirmed","Deaths","Recovered"]` in a `pandas` dataframe given a url to a corresponding csv file:

In [ ]:
#export
def getTimeSeriesConfirmed(download: bool=False, force: bool=False) -> pd.DataFrame:
    url = f'{ROOT}/csse_covid_19_time_series/{CONFIRMED}'
    localfile = 'time_series_19-covid-Confirmed.csv'
    return procUrl(url, download, localfile, force)

def getTimeSeriesDeaths(download: bool=False, force: bool=False) -> pd.DataFrame:
    url = f'{ROOT}/csse_covid_19_time_series/{DEATHS}'
    localfile = 'time_series_19-covid-Deaths.csv'
    return procUrl(url, download, localfile, force)

def getTimeSeriesRecovered(download: bool=False, force: bool=False) -> pd.DataFrame:
    url = f'{ROOT}/csse_covid_19_time_series/{RECOVERED}'
    localfile = 'time_series_19-covid-Recovered.csv'
    return procUrl(url, download, localfile, force)

The following cell contains methods to aggregate each of `["Confirmed","Deaths","Recovered"]` by country.  Note that `force` and `download` are both set `True` in all cases.  Note also that at time of writing `Recovered` isn't supported as a time series dataset in a csv file.

In [ ]:
#export
def procTimeSeriesDataframe(r: List, kind: str, log:bool=True) -> pd.DataFrame:
    sdf = pd.DataFrame(r)
    sdf['day'] = sdf['day'].apply(pd.to_datetime)
    if log:
        sdf[f'Log{kind}'] = np.log(sdf[kind])
    #sdf.set_index('day', drop=True, inplace=True)
    return sdf

def procTimeSeries(df: pd.DataFrame, kind: str) -> pd.DataFrame:
    r = []
    countries = df.groupby('Country/Region')
    cols = df.columns.to_list()
    for country, group in countries:
        total = []
        for row_index, row in group.iterrows():
            rvals = row.to_list()
            if not len(total):
                total = rvals[4:]
                #print('first',total)
            else:
                #print('next',rvals[4:])
                total = [a+b for a, b in zip(total, rvals[4:])]
        for a, b in zip(cols[4:], total):
            r.append({'day':a, 'country':country, kind:b})
    return procTimeSeriesDataframe(r, kind)

procTimeSeriesDeaths: Callable[[pd.DataFrame], pd.DataFrame] = lambda: procTimeSeries(getTimeSeriesDeaths(download=True, force=True), 'Deaths')
procTimeSeriesConfirmed: Callable[[pd.DataFrame], pd.DataFrame] = lambda: procTimeSeries(getTimeSeriesConfirmed(download=True, force=True), 'Confirmed')
procTimeSeriesRecovered: Callable[[pd.DataFrame], pd.DataFrame] = lambda: procTimeSeries(getTimeSeriesRecovered(download=True, force=True), 'Recovered')

The following cell contains a utility method for diffing between any of `["Confirmed","Deaths","Recovered"]` by country to create a `New` column.  This method also adds a log column.

In [ ]:
#export
def procNewCasesTimeSeries(df: pd.DataFrame, kind:str) -> pd.DataFrame:
    r = []
    #sdf = df[df.country.isin(countries)].copy()
    countryGroups = df.groupby('country')
    cols = df.columns.to_list()
    cols.append('New')
    for country, group in countryGroups:
        gdf = group.copy()
        gdf['New'] = gdf[kind].diff()
        for row_index, row in gdf.iterrows():
            r.append(dict(zip(cols,row.to_list())))
    ndf = pd.DataFrame(r)
    ndf['LogNew'] = np.log(ndf['New'])
    return ndf

The following cell contains a utility plotting method for the processed and aggregated time series dataframe.  It has additional support for `altair` to build a log graph using buil-in `alt.Scale` support:

In [ ]:
#export
def plotCountriesTimeSeries(df: pd.DataFrame, countries: List, which: str, x: str, y: str,
                            grid: bool=True, log: bool=False, clampx: int=100, clampy: int=7, 
                            visualisation='matplotlib') -> None:
    if visualisation == 'altair':
        sdf = df[df.country.isin(countries)]
        title = f'{y} by {x} in {countries} as of {which}'
        if x=='day':
            xval = 'day:T'
        else:
            if log:
                xval = alt.X(f'{x}:Q', scale=alt.Scale(type='log', domain=(clampx, 30000)), axis=alt.Axis(tickCount=8))
                sdf = sdf[sdf[x] > clampx]
            else:
                xval = f'{x}:Q'
        if log:
            title = f'Log graph of {y} by {x} in {countries} as of {which}'
            yval = alt.Y(f'{y}:Q', scale=alt.Scale(type='log'), axis=alt.Axis(tickCount=4))
            sdf = sdf[sdf[y] > clampy]
        else:
            yval = f'{y}:Q'
        line_chart = alt.Chart(sdf).mark_line().encode(
            x=xval,
            y=yval,
            color=alt.Color('country:N', scale=alt.Scale(scheme='tableau20'))
        ).properties(
            width=ALTAIR_W,
            height=ALTAIR_H,
            title=title
        )
        if not grid:
            line_chart = removeGrid(line_chart)
        return line_chart
    else:
        fig, ax = plt.subplots()
        for country in countries:
            ax = df[df['country'] == country].plot(ax=ax, x=x, y=y, kind='line', figsize=(18,9))
        ax.set_ylabel(y, size=14)
        ax.set_xlabel('Day', size=14)
        ax.set_title(f'{y} by {x} in {countries} as of {which}', size=18)
        ax.legend(ax.get_lines(),countries)
        plt.show()

## 3. Graphing counts and time series via Covid API

The following cell contains a utility method for plotting a sorted stacked bar graph of country data.  For the `altair` version of the stacked bar graph in `plotCountriesDailyReportFromAPI()`, this requires that we EITHER use [`df.melt()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.melt.html) support to convert a wide format dataframe to a long format one OR we use `altair` `transform_fold()` support to manage that conversion for us.
```
pandas.melt(frame: pandas.core.frame.DataFrame, id_vars=None, value_vars=None, var_name=None, value_name='value', col_level=None) → pandas.core.frame.DataFrame
```

In [ ]:
#export
def getCountriesDailyReportFromAPI(normalised: bool=False) ->  pd.DataFrame:
    url = 'https://api.covid19api.com/summary'
    if normalised:
        df = pd.DataFrame(requests.get(url).json().get('Countries'))
        df.Country.replace({'Iran (Islamic Republic of)': 'Iran', 'Korea, South': 'South Korea'},inplace=True)
        cols = df.columns.to_list()
        sdf = df.groupby('Country')[cols[2:]].apply(sum).reset_index()
        sdf = sdf.sort_values(by=['TotalConfirmed'], ascending=False)
    else:
        sdf = pd.DataFrame(requests.get(url).json().get('Countries')).\
          sort_values(by=['TotalConfirmed'], ascending=False)
    return sdf

def plotCountriesDailyReportFromAPI(normalised=False, grid: bool=True, visualisation='matplotlib', cutoff=10, transformFold=True) -> None:
    sdf = getCountriesDailyReportFromAPI(normalised)
    target = ['TotalRecovered','TotalConfirmed','TotalDeaths']
    targetColors = ['green','orange','red']
    if visualisation == 'altair':
        if transformFold:
            bar_chart = alt.Chart(sdf[sdf.TotalDeaths > cutoff]).transform_fold(
                target,
                as_ = ['Category','Count']
            )
        else:
            sdf2 = sdf[sdf.TotalDeaths > cutoff].melt(id_vars=['Country','Slug'], value_vars=target, var_name='Category', value_name='Count')
            bar_chart = alt.Chart(sdf2)
        bar_chart = bar_chart.mark_bar(size=10).encode(
            x=alt.X('Country',sort='-y'),
            y='Count:Q',
            order=alt.Order(# Sort the segments of the bars by this field
                'Category:N',
                sort='descending'
            ),
            color = alt.Color('Category:N',
                scale = alt.Scale(domain=target, range=targetColors),
                legend = alt.Legend(title="Category")
            )
        ).properties(
            width=ALTAIR_W,
            height=ALTAIR_H,
            title='Covid-19 cases and deaths'
        )
        if not grid:
            bar_chart = removeGrid(bar_chart)
        return bar_chart
    else: # matplotlib
        _ = sdf[sdf.TotalDeaths > cutoff].plot(kind='bar', x='Country', y=target,\
          color=targetColors, stacked=True, figsize=(18, 9)).set_title('Covid-19 cases and deaths', size=18)

def getCategoryByCountryFromAPI(category: str, country: str, color: str) ->  pd.DataFrame:
    url = f'https://api.covid19api.com/total/country/{country}/status/{category.lower()}'
    sdf = pd.DataFrame(requests.get(url).json())
    sdf['Date'] = sdf['Date'].apply(pd.to_datetime)
    sdf.rename(columns={"Cases": category}, inplace=True)
    sdf[f'Log{category}'] = np.log(sdf[category])
    return sdf

def plotCategoryByCountryFromAPI(category:str, country:str, color: str='orange', log: bool=False,
                                 grid: bool=True, visualisation: str='matplotlib') -> None:
    sdf = getCategoryByCountryFromAPI(category, country, color)
    if log:
        category = f'Log{category}'
    if visualisation == 'altair':
        line_chart = alt.Chart(sdf).mark_line().encode(
            x='Date:T',
            y=category,
            color=alt.value(color)
        ).properties(
            width=ALTAIR_W,
            height=ALTAIR_H,
            title=f'Covid-19 {category} in {country}'
        )
        if not grid:
            line_chart = removeGrid(line_chart)
        return line_chart
    else: # matplotlib
        sdf.plot(kind='line', x='Date', y=category, color=color, figsize=(18, 9)).\
          set_title(f'Covid-19 {category} in {country}', size=18)

## 4. Testing

From [the nbdev documentation](http://nbdev.fast.ai/test/):
> Everything that is not an exported cell is considered a test, so you should make sure your notebooks can all run smoothly (and fast) if you want to use this functionality as the CLI. 

In [ ]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
